In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install transformers
!pip install sentencepiece
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.

In [4]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import os
import json

In [53]:
import os
import json
import re
from zipfile import ZipFile

cond = re.compile('[가-힣]+')

def get_data_from_zip(zip_name):
    zip_file_path = '/content/drive/MyDrive/jolnon/' + zip_name + '.zip'
    extract_path = '/content/' + zip_name

    # zip 파일 압축 해제
    with ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    result = []
    for name in os.listdir(path=extract_path):
        file_path = os.path.join(extract_path, name)
        if os.path.isdir(file_path):
            for name in os.listdir(path=file_path):
                with open(file_path + '/' + name, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    valid = []
                    for d in data['content'].split('\n'):
                        res = []
                        for word in d.split(' '):
                            if cond.match(word) != None:
                                res.append(word)
                        d = ' '.join(res)
                        if d and not d.replace(' ', '').isdecimal():
                            valid.append(d)
                    result.append('\n'.join([data['title']] + valid))
        else:
            with open(extract_path + '/' + name, 'r', encoding='utf-8') as f:
                data = json.load(f)
                valid = []
                for d in data['content'].split('\n'):
                    res = []
                    for word in d.split(' '):
                        if cond.match(word) != None:
                            res.append(word)
                    d = ' '.join(res)
                    if d and not d.replace(' ', '').isdecimal():
                        valid.append(d)
                result.append('\n'.join([data['title']] + valid))
    return result


In [54]:
import random
random.seed(42)
galleries = ['animal','tree']#,'tree']#,'ktwiz','kartriderdrift','skwyverns_new1','ncdinos','samsunglions_new','doosanbears_new1','giants_new2','sh_new','lgtwins_new','tigers_new']
dataset = []
for e in galleries:
  dataset.append([get_data_from_zip(e)])
cocktail = []
for data in dataset:
  for sen in data:
    cocktail+=sen

In [55]:
def remove_words(sentence, word):
    return ' '.join([w for w in sentence.split() if w != word])
stopwords = ['-dc', 'official', 'App', '이미지' ,'순서','저장','jpg','png','txt','gif']#['ㅋㅋ', 'ex', '나는', 'on', '이미지', '순서', '오늘', '일단', 'and', '이야', '그리고', '내일', '그냥', '000', '조금', '살짝', 'ㅇㅇ', 'ㅈㄱㄴ', '있음', '이거', '내가', '칵하하하', '칵하핫', '근데', '지듣노', 'youtube ', '야스중', '우흥', '한다', 'ㅎㅎ', 'ㅠㅠ', '로오오오오오옹', '하고', '아침', '것도', '추천', '혹시', '새낀데', '같다']
for i in range(len(cocktail)):
  for j in range(len(stopwords)):
    cocktail[i] = remove_words(cocktail[i],stopwords[j])

In [56]:
import collections

def remove_top_n_words(data, n):
    words = [word for sentence in data for word in sentence.split(' ')]
    word_count = collections.Counter(words)
    top_n_words = [word for word, _ in word_count.most_common(n)]
    for i in range(len(data)):
        for word in top_n_words:
            if word=='':
                continue
            data[i] = remove_words(data[i], word)
    return data

def remove_empty_sentences(cocktail, real_labels):
  indices = [i for i, sentence in enumerate(cocktail) if sentence.strip() != '']
  cocktail = [cocktail[i] for i in indices]
  real_labels = [real_labels[i] for i in indices]
  return cocktail, real_labels


In [57]:
real_labels = []
for i in range(len(dataset)):
  real_labels += [i] * len(dataset[i][0])
cocktail = remove_top_n_words(cocktail, 0)
cocktail, real_labels = remove_empty_sentences(cocktail, real_labels)

In [58]:
from transformers import AutoModel, AutoTokenizer
from umap import UMAP
# umap_model = UMAP(random_state=42)
# umap_model = UMAP(n_neighbors=15, n_components=10, metric='cosine', random_state=42)
# model = AutoModel.from_pretrained("klue/roberta-large")
# tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

# model = AutoModel.from_pretrained("klue/bert-base")
# tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# from kobert_tokenizer import KoBERTTokenizer
# tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# import torch
# from transformers import BertModel
# model = BertModel.from_pretrained('skt/kobert-base-v1')

In [59]:
class CustomTokenizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
  # def __call__(self, target):
  #   return self.tokenizer.tokenize(target)
  def __call__(self, target):
    tokens = self.tokenizer.tokenize(target)
    if len(tokens)==0:
      return []
    if isinstance(tokens[0], str):
      indices = [i for i, token in enumerate(tokens) if not token.startswith('##')]
      tokens = [tokens[i] for i in indices]
    elif isinstance(tokens[0][0], str):
      for j in range(len(tokens)):
        indices = [i for i, token in enumerate(tokens[j]) if not token.startswith('##')]
        tokens[j] = [tokens[j][i] for i in indices]
    return tokens

In [60]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# custom_tokenizer = CustomTokenizer(tokenizer)
# # vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
# vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [61]:
# from umap import UMAP
# from hdbscan import HDBSCAN

# umap_model = UMAP(n_neighbors=5, n_components=5, metric='cosine', low_memory=False)
# hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', prediction_data=True)

In [62]:
from bertopic.vectorizers import ClassTfidfTransformer
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
bertopic_model_embed_token = BERTopic(language='multilingual',
                          nr_topics=100,
                          # ctfidf_model=ctfidf_model,
                          top_n_words=1)
# bertopic_model_embed_token = BERTopic(embedding_model=model,
#                                       vectorizer_model=vectorizer,
#                                       ctfidf_model=ctfidf_model,
#                                       nr_topics=100,
#                                       top_n_words=1,
#                                       calculate_probabilities=False)
topics, _ = bertopic_model_embed_token.fit_transform(cocktail)

In [63]:
topic_info = bertopic_model_embed_token.get_topic_info()
number_of_topics = len(topic_info) - 1
print(f"생성된 주제의 수: {number_of_topics}")
lst = []
for index, row in topic_info.iterrows():
    topic_num = row['Topic']
    if topic_num != -1:
        topic_name = row['Name']
        pure_topic_name = "_".join(topic_name.split("_")[1:])
        lst.append(pure_topic_name)
for i in lst:
  print(i)

생성된 주제의 수: 99
햄스터
로보쿤
데로닉스
꽃이
신엽
물을
이거
어디서
햇빛
잎이
로보쿤
쥐친구
식물
고양이
병원
식물등
정상임
이름이
귀엽다
jpg
플로리다
미분
사료
흙은
거미줄
핸들링
잠자는쥐
이름이
냄새
새벽에
과일트럭에서
나무
코코넛
터널
우리집
마시는
맛있음
모래
뿌리가
서울
김나나
케이지
토마토
쥐가
식쇼
화장실
비둘기
입제
뚱땡정글
식물
유혈목이
집에
기니피그
애완동물
일베
수컷이
알로카시아
고추
뽀뽀
딸기
사진
올리브
선반을
토끼들
생태계교란
얼음
유통기한
블루베리
유튜브
빨리
난건지
자는
냉장고를
나무위키
미모
강아지
돼지
뿌리
ㅇㅇ
멍청햄
천리향
개구리
제라늄
거북이
알파카한테
표범
전면부를
철물점
김나나
버섯이
어디임
금방
미쳤나봐
피는지
행잉
아몬드
곰스터가
엄마한테
물고기콘


In [64]:
table = [[0 for r in range(2)] for t in range(number_of_topics)]

for j in range(number_of_topics):
  for i in range(len(topics)):
    if topics[i]==j:
      table[j][real_labels[i]]+=1
print(table)

[[1502, 2], [564, 392], [333, 222], [11, 530], [91, 280], [23, 285], [102, 180], [54, 130], [57, 115], [1, 162], [98, 17], [108, 1], [0, 102], [85, 15], [85, 11], [1, 94], [40, 53], [12, 80], [70, 20], [86, 3], [20, 63], [29, 52], [66, 15], [2, 76], [14, 63], [69, 7], [68, 3], [0, 70], [37, 30], [51, 12], [40, 19], [4, 55], [43, 12], [29, 23], [40, 9], [23, 25], [35, 12], [42, 3], [0, 43], [13, 30], [5, 36], [38, 3], [1, 40], [40, 0], [12, 25], [33, 4], [34, 1], [3, 31], [30, 3], [1, 30], [28, 1], [25, 3], [27, 1], [25, 2], [26, 0], [25, 0], [1, 23], [0, 22], [20, 2], [5, 16], [20, 0], [0, 20], [11, 9], [20, 0], [18, 2], [1, 18], [5, 14], [4, 15], [12, 7], [0, 17], [9, 8], [16, 0], [16, 0], [14, 2], [0, 16], [13, 3], [14, 1], [0, 15], [14, 1], [13, 1], [0, 14], [9, 4], [0, 13], [11, 2], [10, 3], [7, 6], [0, 13], [4, 8], [11, 1], [0, 12], [6, 6], [0, 11], [0, 11], [2, 9], [4, 7], [10, 1], [10, 0], [10, 0], [7, 3]]


In [65]:
### Reducing outliers
topics = bertopic_model_embed_token.reduce_outliers(cocktail, topics, strategy="c-tf-idf", threshold=0.1)
topics = bertopic_model_embed_token.reduce_outliers(cocktail, topics, strategy="distributions")

In [66]:
# import matplotlib.pyplot as plt

# colors = ["#0000FF", "#FF0000"]

# data = [[] for _ in range(len(dataset))]
# for j in range(len(dataset)):
#     data[j] = [topics[i] for i in range(len(real_labels)) if real_labels[i] == j]
#     # plt.hist(data[j], bins=number_of_topics, color=colors[j], histtype='bar')
#     # plt.show()

# plt.hist(data, bins=number_of_topics, color=colors, histtype='bar', density=True)
# plt.show()

In [67]:
# import numpy as np
# npdata = [[] for _ in range(len(dataset))]
# for j in range(len(dataset)):
#   npdata[j] = np.array([0] * (number_of_topics + 1), dtype=np.float32)
#   for k in range(len(data[j])):
#     npdata[j][data[j][k] + 1] += 1
#   npdata[j] /= npdata[j].sum()

In [68]:
# topic_ = [[], []]
# threshold = 5
# for k in range(number_of_topics):
#   ratio = max((npdata[0][k] + 1e-12) / (npdata[1][k] + 1e-12), (npdata[1][k] + 1e-12) / (npdata[0][k] + 1e-12))
#   if ratio > threshold:
#     if npdata[0][k] > npdata[1][k]:
#       topic_[0].append((k - 1, npdata[0][k], npdata[1][k]))
#     else:
#       topic_[1].append((k - 1, npdata[0][k], npdata[1][k]))

# for k in range(len(topic_)):
#   topic_[k].sort(key=lambda x: x[k + 1], reverse=True)
#   for j, v0, v1 in topic_[k]:
#     print('\t'.join([x[0] for x in bertopic_model_embed_token.get_topic(j)]), j, v0, v1, sep='\t')
#   print()

In [69]:
# for k in range(len(npdata)):
#   data = sorted([(i - 1, v) for i, v in enumerate(npdata[k].tolist())], key=lambda x: x[1], reverse=True)
#   for i in range(10):
#     print('\t'.join([x[0] for x in bertopic_model_embed_token.get_topic(data[i][0])]), data[i][0], data[i][1], sep='\t')
#   print()

In [70]:
hierarchical_topics = bertopic_model_embed_token.hierarchical_topics(cocktail)

100%|██████████| 98/98 [00:02<00:00, 48.01it/s]


In [71]:
# Hierarchical topics
# linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
# hierarchical_topics = bertopic_model_embed_token.hierarchical_topics(cocktail, linkage_function=linkage_function)

In [72]:
bertopic_model_embed_token.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [73]:
hierarchical_topics

,Parent_ID,Parent_Name,Topics,Child_Left_ID,Child_Left_Name,Child_Right_ID,Child_Right_Name,Distance
97,196,햄스터,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",194,식물,195,햄스터,1.480400
96,195,햄스터,"[0, 1, 2, 10, 11, 13, 14, 18, 19, 22, 25, 26, ...",184,햄스터,193,jpg,1.288572
95,194,식물,"[3, 4, 5, 6, 7, 8, 9, 12, 15, 16, 17, 20, 21, ...",122,이름이,190,꽃이,1.275225
94,193,jpg,"[1, 2, 10, 11, 13, 18, 19, 26, 30, 33, 34, 35,...",192,로보쿤,168,jpg,1.147435
93,192,로보쿤,"[1, 2, 10, 13, 18, 26, 30, 33, 34, 35, 36, 39,...",187,로보쿤,191,고양이,1.121504
...,...,...,...,...,...,...,...,...
4,103,꽃이,"[3, 4, 5, 9]",100,꽃이,101,잎이,0.818805
3,102,햄스터,"[0, 25]",0,햄스터,25,핸들링,0.769540
2,101,잎이,"[5, 9]",5,물을,9,잎이,0.738244
1,100,꽃이,"[3, 4]",4,신엽,3,꽃이,0.672649


In [74]:
bertopic_model_embed_token.visualize_heatmap()

In [75]:
tree = bertopic_model_embed_token.get_topic_tree(hierarchical_topics)

In [76]:
print(tree)

.
├─식물
│    ├─이름이
│    │    ├─■──나무 ── Topic: 31
│    │    └─이름이
│    │         ├─■──이름이 ── Topic: 27
│    │         └─■──이름이 ── Topic: 17
│    └─꽃이
│         ├─꽃이
│         │    ├─식물
│         │    │    ├─식물
│         │    │    │    ├─햇빛
│         │    │    │    │    ├─■──천리향 ── Topic: 80
│         │    │    │    │    └─■──햇빛 ── Topic: 8
│         │    │    │    └─식물
│         │    │    │         ├─식물
│         │    │    │         │    ├─■──식물등 ── Topic: 15
│         │    │    │         │    └─■──식물 ── Topic: 12
│         │    │    │         └─■──식물 ── Topic: 49
│         │    │    └─토마토
│         │    │         ├─거미줄
│         │    │         │    ├─거미줄
│         │    │         │    │    ├─■──정상임 ── Topic: 16
│         │    │         │    │    └─거미줄
│         │    │         │    │         ├─■──미분 ── Topic: 21
│         │    │         │    │         └─■──거미줄 ── Topic: 24
│         │    │         │    └─■──전면부를 ── Topic: 86
│         │    │         └─토마토
│         │    │              ├─